## Data Preprocessing

In [1]:
# Data = CMAPSSData
# Source = https://ti.arc.nasa.gov/tech/dash/groups/pcoe/prognostic-data-repository/
# Dataset = Turbofan Engine Degradation Simulation Data Set

In [2]:
# Data to be used - Train and Test data for 1

In [3]:
import pandas as pd
import numpy as np
import pickle as pkl
import matplotlib.pyplot as plt

### Notebook options

In [4]:
pd.set_option('display.max_colwidth', -1) # Column width
plt.rcParams['figure.figsize'] = [21, 10] # Size of the plots

### Data Directories

In [5]:
INPUT_DIR = '../data/raw/CMAPSSData/'
INT_DIR = '../data/interim/'
OUTPUT_DIR = '../data/interim/'

### Read the input data

In [8]:
input_file = OUTPUT_DIR + 'train_case3.csv'
df_data = pd.read_csv(input_file)

### Data Imputation

In [9]:
# Not required as there are no missing values
# Remove the additional columns

### Data Standardization

In [10]:
df_data.head()

,unit,time_cycles,setting1,setting2,setting3,meas01,meas02,meas03,meas04,meas05,...,meas12,meas13,meas14,meas15,meas16,meas17,meas18,meas19,meas20,meas21
0,1,1,-0.0005,0.0004,100.0,518.67,642.36,1583.23,1396.84,14.62,...,522.31,2388.01,8145.32,8.4246,0.03,391,2388,100.0,39.11,23.3537
1,1,2,0.0008,-0.0003,100.0,518.67,642.50,1584.69,1396.89,14.62,...,522.42,2388.03,8152.85,8.4403,0.03,392,2388,100.0,38.99,23.4491
2,1,3,-0.0014,-0.0002,100.0,518.67,642.18,1582.35,1405.61,14.62,...,522.03,2388.00,8150.17,8.3901,0.03,391,2388,100.0,38.85,23.3669
3,1,4,-0.0020,0.0001,100.0,518.67,642.92,1585.61,1392.27,14.62,...,522.49,2388.08,8146.56,8.3878,0.03,392,2388,100.0,38.96,23.2951
4,1,5,0.0016,0.0000,100.0,518.67,641.68,1588.63,1397.65,14.62,...,522.58,2388.03,8147.80,8.3869,0.03,392,2388,100.0,39.14,23.4583


In [12]:
from sklearn.preprocessing import StandardScaler

exclude_cols = ['unit', 'time_cycles']

df_pca = df_data.copy()
features = [x for x in df_pca.columns.values if x not in set(exclude_cols)]
print(features)

# Separating out the features
x = df_pca.loc[:, features].values

# Standardizing the features
x = StandardScaler().fit_transform(x)

['setting1', 'setting2', 'setting3', 'meas01', 'meas02', 'meas03', 'meas04', 'meas05', 'meas06', 'meas07', 'meas08', 'meas09', 'meas10', 'meas11', 'meas12', 'meas13', 'meas14', 'meas15', 'meas16', 'meas17', 'meas18', 'meas19', 'meas20', 'meas21']


In [13]:
df_pca.head()

,unit,time_cycles,setting1,setting2,setting3,meas01,meas02,meas03,meas04,meas05,...,meas12,meas13,meas14,meas15,meas16,meas17,meas18,meas19,meas20,meas21
0,1,1,-0.0005,0.0004,100.0,518.67,642.36,1583.23,1396.84,14.62,...,522.31,2388.01,8145.32,8.4246,0.03,391,2388,100.0,39.11,23.3537
1,1,2,0.0008,-0.0003,100.0,518.67,642.50,1584.69,1396.89,14.62,...,522.42,2388.03,8152.85,8.4403,0.03,392,2388,100.0,38.99,23.4491
2,1,3,-0.0014,-0.0002,100.0,518.67,642.18,1582.35,1405.61,14.62,...,522.03,2388.00,8150.17,8.3901,0.03,391,2388,100.0,38.85,23.3669
3,1,4,-0.0020,0.0001,100.0,518.67,642.92,1585.61,1392.27,14.62,...,522.49,2388.08,8146.56,8.3878,0.03,392,2388,100.0,38.96,23.2951
4,1,5,0.0016,0.0000,100.0,518.67,641.68,1588.63,1397.65,14.62,...,522.58,2388.03,8147.80,8.3869,0.03,392,2388,100.0,39.14,23.4583


### Writing standardized to file

In [14]:
std_file = OUTPUT_DIR + 'train_case1_std.pkl'
with open(std_file, 'wb') as f:
    pkl.dump(x, f, protocol=pkl.HIGHEST_PROTOCOL)

### Reading back from pickle

In [15]:
with open(std_file, 'rb') as f:
    x = pkl.load(f)

### PCA Projection

In [16]:
from sklearn.decomposition import PCA
num_components = len(df_pca.columns.values) - 2
pca = PCA(n_components=num_components)
principalComponents = pca.fit_transform(x)

In [17]:
sum = 0
idx = 1
print('| Component | Individual Contribution | Cumulative |')
print('| -- | -- | -- |')
for val in pca.explained_variance_ratio_:
    sum += val
    print('| Component ' + str(idx), end = ' | ')
    print("{:10.2f}".format(val*100), end=' | ')
    print("{:10.2f}".format(sum*100), end= ' |\n')
    idx += 1

| Component | Individual Contribution | Cumulative |
| -- | -- | -- |
| Component 1 |      41.38 |      41.38 |
| Component 2 |      26.37 |      67.75 |
| Component 3 |       5.58 |      73.33 |
| Component 4 |       5.56 |      78.89 |
| Component 5 |       5.13 |      84.02 |
| Component 6 |       4.34 |      88.36 |
| Component 7 |       2.40 |      90.76 |
| Component 8 |       1.88 |      92.64 |
| Component 9 |       1.70 |      94.34 |
| Component 10 |       1.39 |      95.73 |
| Component 11 |       1.12 |      96.84 |
| Component 12 |       0.89 |      97.73 |
| Component 13 |       0.77 |      98.50 |
| Component 14 |       0.70 |      99.20 |
| Component 15 |       0.34 |      99.53 |
| Component 16 |       0.21 |      99.74 |
| Component 17 |       0.20 |      99.94 |
| Component 18 |       0.06 |     100.00 |
| Component 19 |       0.00 |     100.00 |
| Component 20 |       0.00 |     100.00 |
| Component 21 |       0.00 |     100.00 |
| Component 22 |       0.00 |     10

### Create a principal dataframe with all the components

In [18]:
pca_col_list = ['pc'+str(i+1) for i in range(num_components)]
principalDf = pd.DataFrame(data = principalComponents, columns = pca_col_list)

### Find the influence of features in each component

In [19]:
df_infl = pd.DataFrame(pca.components_,columns=features,index=pca_col_list)

In [20]:
df_infl

,setting1,setting2,setting3,meas01,meas02,meas03,meas04,meas05,meas06,meas07,...,meas12,meas13,meas14,meas15,meas16,meas17,meas18,meas19,meas20,meas21
pc1,-2.085383e-03,1.432240e-03,1.110223e-16,2.775558e-17,2.007096e-01,2.458672e-01,2.294103e-01,-0.000000e+00,2.339025e-02,3.012531e-01,...,3.057597e-01,3.117744e-01,2.816727e-01,-1.731526e-01,-0.000000e+00,2.529474e-01,-0.000000e+00,-0.0,1.655271e-01,1.677629e-01
pc2,1.412365e-03,1.976861e-03,-3.642919e-17,1.110223e-16,2.905410e-01,2.293067e-01,2.954265e-01,0.000000e+00,2.699801e-01,-2.435912e-01,...,-2.359161e-01,8.950946e-02,-2.027811e-02,3.779747e-01,0.000000e+00,2.403067e-01,0.000000e+00,0.0,-3.693626e-01,-3.692227e-01
pc3,-5.949306e-02,9.681333e-01,9.714451e-17,3.295975e-17,6.163016e-03,1.570609e-03,6.276187e-03,-2.220446e-16,-9.643710e-02,-2.387594e-02,...,-2.202118e-02,-8.122030e-02,1.371296e-01,6.295483e-03,-0.000000e+00,-1.377298e-03,-0.000000e+00,-0.0,-2.727350e-02,-2.858743e-02
pc4,9.950620e-01,7.655044e-02,-1.387779e-17,0.000000e+00,-1.163485e-03,-2.007247e-03,2.409231e-03,-8.239937e-18,1.262933e-02,8.636138e-03,...,7.751478e-03,2.055021e-02,-3.787903e-02,-2.079442e-04,0.000000e+00,5.965849e-03,0.000000e+00,0.0,5.653472e-03,9.881174e-03
pc5,7.704745e-02,-2.376649e-01,2.775558e-16,5.551115e-17,3.783330e-02,7.270527e-03,4.848516e-02,2.775558e-17,-4.128787e-01,-8.471069e-02,...,-8.110976e-02,-3.387421e-01,5.112637e-01,2.082428e-02,-0.000000e+00,1.204499e-02,-0.000000e+00,-0.0,-1.262942e-01,-1.163628e-01
pc6,-1.659121e-02,1.538238e-02,-2.220446e-16,-2.151057e-16,2.159252e-01,1.190866e-01,2.074491e-01,5.551115e-17,-7.038730e-01,1.001363e-02,...,1.374884e-02,-6.301656e-02,-3.943318e-01,-1.004301e-01,-0.000000e+00,1.055387e-01,-0.000000e+00,-0.0,-6.266141e-02,-6.131332e-02
pc7,1.705671e-03,9.055215e-03,-1.942890e-16,-9.714451e-17,3.061818e-01,-1.072757e-01,1.753775e-01,-1.665335e-16,4.532112e-01,1.100922e-01,...,1.130216e-01,-4.455350e-01,-1.046986e-01,1.522925e-02,-0.000000e+00,-6.575331e-02,-0.000000e+00,-0.0,1.040932e-01,1.142095e-01
pc8,3.995168e-03,-1.422305e-04,2.775558e-16,-1.387779e-17,-3.523474e-01,8.744020e-01,-1.221429e-02,1.110223e-16,7.712295e-02,5.146526e-03,...,6.064479e-03,-1.321626e-01,-3.798255e-02,-1.965378e-03,-0.000000e+00,-2.505473e-01,-0.000000e+00,-0.0,2.274226e-02,1.941419e-02
pc9,4.791765e-03,-4.893685e-03,-1.110223e-16,0.000000e+00,7.097728e-01,1.600013e-01,-1.543561e-01,1.110223e-16,-5.522115e-02,-1.623332e-03,...,-4.815912e-03,8.661797e-02,4.663376e-02,-4.905289e-02,0.000000e+00,-5.937005e-01,0.000000e+00,0.0,4.697681e-02,5.535270e-02
pc10,-1.646016e-03,-1.884247e-03,0.000000e+00,-6.245005e-17,2.805007e-01,2.478956e-01,-3.252382e-01,-1.526557e-16,2.911479e-02,1.941625e-02,...,1.971000e-02,-1.771982e-01,-4.886112e-02,-9.793808e-02,0.000000e+00,6.452661e-01,0.000000e+00,0.0,2.157077e-01,1.689953e-01


### Write the principal components to file

In [21]:
principalDf_file = INT_DIR + 'fd003_components_' + str(num_components) + '.csv'
with open(principalDf_file, 'wb') as f:
    principalDf.to_csv(principalDf_file, header=True, index=False)

print('Writing to ', principalDf_file)

Writing to  ../data/interim/fd003_components_24.csv
